In [1]:
import matplotlib.pyplot as plt
import numpy as np_normal
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from memory_exp import *
from manual_models import *

In [2]:
def coherence(phi):
    m, n = phi.shape
    a = phi
    a = phi /  np.linalg.norm(phi, axis=0)
    a_ = np.matmul(a.T.conj(), a)
    a = np.abs(a_ - np.eye(a_.shape[0]))
    return np.max(a, axis=None)


In [3]:
def welch(T,d):
    assert T > d
    return np.sqrt((T-d)/(d*(T-1)))

data = [
    (7, 3, [1, 2, 4]),
    (7, 4, [0, 3, 5, 6]),
    (13, 4, [0, 1, 3, 9]),
    (11, 5, [1, 3, 4, 5, 9]),
    (21, 5, [3, 6, 7, 12, 14]),
    (11, 6, [0, 2, 6, 7, 8, 10]),
    (31, 6, [1, 5, 11, 24, 25, 27]),
    (15, 7, [0, 1, 2, 4, 5, 8, 10]),
    (15, 8, [3, 6, 7, 9, 11, 12, 13, 14]),
    (57, 8, [1, 6, 7, 9, 19, 38, 42, 49]),
    (13, 9, [2, 4, 5, 6, 7, 8, 10, 11, 12]),
    (37, 9, [1, 7, 9, 10, 12, 16, 26, 33, 34]),
    (73, 9, [1, 2, 4, 8, 16, 32, 37, 55, 64]),
    (40, 13, [0, 1, 3, 5, 9, 15, 22, 25, 26, 27, 34, 35, 38])
]
for T,d,_ in data:
    print(T*2,d*2, welch(T=T,d=d), 1/welch(T=T,d=d)**2)

14 6 0.4714045207910317 4.5
14 8 0.3535533905932738 7.999999999999998
26 8 0.4330127018922193 5.333333333333334
22 10 0.34641016151377546 8.333333333333334
42 10 0.4 6.249999999999999
22 12 0.28867513459481287 12.0
62 12 0.37267799624996495 7.199999999999999
30 14 0.2857142857142857 12.250000000000002
30 16 0.25 16.0
114 16 0.33071891388307384 9.142857142857142
26 18 0.19245008972987526 26.999999999999996
74 18 0.2939723678960656 11.571428571428573
146 18 0.31426968052735443 10.125000000000002
80 26 0.23076923076923078 18.777777777777775


In [9]:
import numpy as np

def fft_matrix(N):
    # Initialize an N x N matrix
    F_N = np.zeros((N, N), dtype=np.complex128)

    # Fill the matrix with the FFT coefficients
    for j in range(N):
        for k in range(N):
            F_N[j, k] = np.exp(2j * np.pi * j * k / N)

    # Return the FFT matrix
    return F_N

# Example usage:
N, k, cols = 15, 7, [0, 1, 2, 4, 5, 8, 10]
embd = fft_matrix(N)#.T#.real
embd = embd[:,cols] * 1/np.sqrt(k)
#plt.imshow(embd)
embd = embd #/ np.linalg.norm(embd,keepdims=True,axis=-1)
print(embd.shape)
overlap_matrix = (embd @ embd.conj().T)#.real.T

def complex_to_real_matrix(A):
    # Separate the real and imaginary parts
    Re_A = np.real(A)
    Im_A = np.imag(A)

    # Construct the new real matrix
    top = np.hstack((Re_A, -Im_A))
    bottom = np.hstack((Im_A, Re_A))
    B = np.vstack((top, bottom))

    return B

embd = complex_to_real_matrix(embd)
print(embd.shape)

(15, 7)
(30, 14)


In [10]:

embd = embd[:,-T:]
current_phi = embd

In [11]:
T = 30
L = 10
d = 14

config = {
    'T': T,
    'model_dim': d,
    'p': T,
    'seq_len':L,
    'attention_input': 'only_sem',
    'no_softmax': True,
    'dataset_type': 'backward'
}


In [12]:
has_BOS =  'BOS' in config['dataset_type']
L = config['seq_len']
d = config['model_dim']
model = TransformerSeq2Seq(T=config['T']+int(has_BOS),
                            model_dim=config['model_dim'],
                            p=config['p'],n_classes=config['seq_len']+1,
                            L=config['seq_len'],
                            attention_input=config['attention_input'],
                            use_softmax=not config['no_softmax']).to(device)

In [13]:
word_embeddings_basis = current_phi# generate_orthogonal_vectors(config['model_dim']) #current_phi.T#

word_embeddings = word_embeddings_basis
model.semantic_emb.weight.data = torch.tensor(word_embeddings).float().to(device)

model.token_mixer.Q.data = 1/L * torch.eye(config['model_dim']).to(device) * np.sqrt(config['model_dim'])
model.token_mixer.K.data = torch.eye(config['model_dim']).to(device)

softmax_table = []
for i in range(1,config['seq_len']+1):
    x = [0] * i + (L-i) * [1]
    x = torch.tensor(x ).to(device).reshape(1,-1)
    # load attn matrix
    r = model(x)
    attn_probs = model.attn_probs.cpu().detach().numpy()
    softmax_table.append(attn_probs[0,0,0]*i)

W_1 = torch.tensor(word_embeddings).float().to(device)
b_1 = -1.0
model.fc1.weight.data = torch.tensor(W_1).float().to(device)
model.fc1.bias.data = torch.tensor(b_1).float().to(device)

softmax_table = np.array(softmax_table)
descision_boundaries = softmax_table[:-1] + (softmax_table[1:] - softmax_table[:-1]) -0.01# + (softmax_table[1:] - softmax_table[:-1]) * 0.001
print(descision_boundaries)
ws, bs = implement_W2(descision_boundaries)
# weight 0 is always smallest
ws = np.array([-100] + list(ws))
bs = np.array([-100] + list(bs))
W_2, b_2 = ws, bs
W_2 = W_2[:,None].repeat(config['T'],1)
model.fc2.weight.data = torch.tensor(W_2).float().to(device)
model.fc2.bias.data = torch.tensor(b_2).float().to(device)


[0.19       0.29       0.39000001 0.49000001 0.59000001 0.69000001
 0.79000001 0.89000001 0.99000001]


<ipython-input-13-1b3bd76d99c8>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model.fc1.weight.data = torch.tensor(W_1).float().to(device)


In [14]:
def test(model, T , L, has_BOS = True, verbose=False):
    works = True
    res = []
    failiures = 0
    for i in range(1000):
        for k in range(1,L+1):

            a = np.random.randint(1,T)
            b = np.random.randint(1,T-1)
            if a == b:
                b = T-1
            if has_BOS:
                x = [T] + [a] * k + (L-k) * [b]
            else:
                x = [a] * k + (L-k) * [b]
            x = torch.tensor(x ).to(device).reshape(1,-1)
            r = model(x)
            #print(model.attn_scores)

            res.append((k,model.b[0,0,a].item()))
            if has_BOS:
                #print(r[0,1])
                x_hat = r[0,1].argmax().item()
            else:
                x_hat = r[0,0].argmax().item()
            if x_hat != k:
                if verbose:
                    #print(model.b)
                    print(f"I got {x_hat}, but should have been {k}.")
                    failiures += 1
                works = False

    print('errors ', failiures/(1000*L))
    if works:
        print("Test passed, model works!")
    else:
        print(":((( Test failed, model does not work!")

    return res

In [15]:

res = test(model, T, L, has_BOS='BOS' in config['dataset_type'], verbose=True)

errors  0.0
Test passed, model works!


In [16]:
coherence(current_phi.T)

0.28571428571428653

In [17]:
import jax.numpy as jnp
import jax
from jax import grad, jit
import matplotlib.pyplot as plt
import numpy as np_normal
%matplotlib inline
import numpy as np

In [18]:
@jit
def coherence(phi, _=None):
    m, n = phi.shape
    a = phi
    a = phi /  jnp.linalg.norm(phi, axis=0)
    a_ = jnp.matmul(a.T.conj(), a)
    a = jnp.abs(a_ - jnp.eye(a_.shape[0]))
    return jnp.max(a, axis=None)

@jit
def soft_coherence(phi,inv_temp=15):
    m, n = phi.shape
    a = phi
    a = phi /  jnp.linalg.norm(phi, axis=0)
    a_ = jnp.matmul(a.T.conj(), a)
    a = jnp.abs(a_ - jnp.eye(a_.shape[0]))
    return jnp.mean(jnp.exp(a * inv_temp))

grad_coherence = jit(grad(coherence))
grad_soft_coherence = jit(grad(soft_coherence))

In [19]:
def optimize(phi, steps, grad_fn):
    coherences = []
    soft_coherences = []
    kappa = 0.1
    for i in range(steps):
        kappa += 1/steps
        coh = coherence(phi)
        soft_coh = soft_coherence(phi,kappa)
        g = grad_soft_coherence(phi + np_normal.random.normal(0, 0.00001, size=phi.shape))
        phi = phi - g
        if i%100==0:
            #print(i, coh, soft_coh)
            pass
    coherences.append(coherence(phi))
    return phi, coherences, soft_coherences



In [20]:
d=12
T=32

In [21]:
current_coherence = np.inf
current_phi = None
for i in range(100):
  phi = np_normal.random.randn(d, T)
  phi_new , coherences, _ = optimize(phi, 5000, grad_coherence); # _soft
  print(coherences[-1])
  if coherences[-1] < current_coherence:
    current_phi = phi_new
    current_coherence = coherences[-1]

current_phi = current_phi /  jnp.linalg.norm(current_phi, axis=0)

0.40883902
0.33624908
0.38297036
0.3092438
0.3361281
0.34776464
0.36798733
0.31491444
0.39204502
0.3586696
0.4403425
0.38895074
0.36322242
0.31081682
0.32563943
0.36015484
0.3447719
0.36414498
0.32182646
0.38121673
0.3715696
0.37212643
0.35196003
0.42913303
0.36856994
0.38486266
0.3981856
0.3334132
0.3053283
0.43559226
0.49907112
0.37238127
0.3621313
0.32590637
0.37581578
0.45462912
0.34916463
0.3946003
0.316701
0.42596495
0.39702222
0.49741247
0.36408168
0.4338788
0.31539893
0.39504975
0.37330562
0.3921702
0.31907994
0.4909868
0.31412703
0.36028075
0.37955633
0.31986406
0.32136115
0.42047858
0.30536643
0.4238991
0.31958866
0.3288729
0.35326904
0.3917241
0.3548895
0.32975096
0.35697678
0.38738036
0.50631773
0.3789734
0.3304639
0.4100708
0.32235295
0.3181391
0.40569332
0.35960707
0.36470845
0.324301
0.36265236
0.46055776
0.39843872
0.38193977
0.33856222
0.36727798
0.3682106
0.38466957
0.39068574
0.3686409
0.2994248
0.3298288
0.33990058
0.32348332
0.47247487
0.30197197
0.34793928
0.36847

In [23]:
current_coherence

Array(0.2994248, dtype=float32)

In [24]:
current_phi = np.array(current_phi)

In [25]:
T = 32
L = 10 #(works until L=12)
d = 12 # ok with mc = 0.33

config = {
    'T': T,
    'model_dim': d,
    'p': T,
    'seq_len':L,
    'attention_input': 'only_sem',
    'no_softmax': True,
    'dataset_type': 'backward'
}


In [26]:
has_BOS =  'BOS' in config['dataset_type']
L = config['seq_len']
d = config['model_dim']
model = TransformerSeq2Seq(T=config['T']+int(has_BOS),
                            model_dim=config['model_dim'],
                            p=config['p'],n_classes=config['seq_len']+1,
                            L=config['seq_len'],
                            attention_input=config['attention_input'],
                            use_softmax=not config['no_softmax']).to(device)

In [27]:
word_embeddings_basis = current_phi.T# generate_orthogonal_vectors(config['model_dim']) #current_phi.T#

word_embeddings = word_embeddings_basis
model.semantic_emb.weight.data = torch.tensor(word_embeddings).float().to(device)

model.token_mixer.Q.data = 1/L * torch.eye(config['model_dim']).to(device) * np.sqrt(config['model_dim'])
model.token_mixer.K.data = torch.eye(config['model_dim']).to(device)

softmax_table = []
for i in range(1,config['seq_len']+1):
    x = [0] * i + (L-i) * [1]
    x = torch.tensor(x ).to(device).reshape(1,-1)
    # load attn matrix
    r = model(x)
    attn_probs = model.attn_probs.cpu().detach().numpy()
    softmax_table.append(attn_probs[0,0,0]*i)

W_1 = torch.tensor(word_embeddings).float().to(device)
b_1 = -1.0
model.fc1.weight.data = torch.tensor(W_1).float().to(device)
model.fc1.bias.data = torch.tensor(b_1).float().to(device)

softmax_table = np.array(softmax_table)
descision_boundaries = softmax_table[:-1] + (softmax_table[1:] - softmax_table[:-1]) -0.01# + (softmax_table[1:] - softmax_table[:-1]) * 0.001
print(descision_boundaries)
ws, bs = implement_W2(descision_boundaries)
# weight 0 is always smallest
ws = np.array([-100] + list(ws))
bs = np.array([-100] + list(bs))
W_2, b_2 = ws, bs
W_2 = W_2[:,None].repeat(config['T'],1)
model.fc2.weight.data = torch.tensor(W_2).float().to(device)
model.fc2.bias.data = torch.tensor(b_2).float().to(device)


[0.18999999 0.28999998 0.38999998 0.48999997 0.58999996 0.68999996
 0.78999995 0.88999995 0.98999994]


<ipython-input-27-0acd37579640>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model.fc1.weight.data = torch.tensor(W_1).float().to(device)


In [28]:
res = test(model, T, L, has_BOS='BOS' in config['dataset_type'], verbose=True)

errors  0.0
Test passed, model works!
